In [ ]:
%%capture
%pip install gurobipy
import warnings
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import Model
from gurobipy import GRB
warnings.filterwarnings('ignore')

In [ ]:
# Question 4

# Creating a new model
model = Model("Calm_Candles")

# Decision variables
x_A = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_A")
x_B = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_B")
x_C = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_C")
x_D = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_D")
x_E = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x_E")

# Objective function (maximize profit)
model.setObjective(10*x_A + 5.5*x_B + 10.2*x_C + 4.5*x_D + 6.8*x_E, GRB.MAXIMIZE)

# Constraints
model.addConstr(226*x_A + 113*x_B + 226*x_C + 113*x_D + 140*x_E <= 6000, "Wax")
model.addConstr(6.8*x_A + 3.4*x_B + 6.8*x_C + 3.4*x_D + 4.25*x_E <= 200, "Fragrance")
model.addConstr(13*x_A + 6.5*x_B + 18*x_C + 9*x_D + 24*x_E <= 500, "Wick")
model.addConstr(15*x_A + 8*x_B + 8*x_C + 7.5*x_D + 7.5*x_E <= 250, "DisplaySpace")
model.addConstr(15*x_A >= 60, "MinDisplay_A")
model.addConstr(8*x_C >= 60, "MinDisplay_C")
model.addConstr(7.5*x_E >= 30, "MinDisplay_E")
model.addConstr(x_A + x_B >= x_C + x_D, "Balance")

# Optimizing the model
model.optimize()

# Printing the results
if model.status == GRB.OPTIMAL:
    print("Optimal Allocation:")
    print(f"x_A: {x_A.x:.2f}")
    print(f"x_B: {x_B.x:.2f}")
    print(f"x_C: {x_C.x:.2f}")
    print(f"x_D: {x_D.x:.2f}")
    print(f"x_E: {x_E.x:.2f}")
    print(f"Maximal Profit: ${model.objVal:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 5 columns and 27 nonzeros
Model fingerprint: 0x857fe487
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+03]
Presolve removed 3 rows and 0 columns
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8197876e+02   2.021575e+02   0.000000e+00      0s
       3    2.3360000e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.336000000e+02
Optimal Allocation:
x_A: 4.00
x_B: 8.00
x_C: 12.00
x_D: 0.00
x_E: 4.00
Maximal Profit: $233.60


In [ ]:
# Question 5

# Sets
warehouses = ['W1', 'W2', 'W3']
stores = ['A', 'B', 'C', 'D']

# Supply capacities
supply = {'W1': 70, 'W2': 50, 'W3': 60}

# Demand requirements
demand = {'A': 40, 'B': 30, 'C': 50, 'D': 60}

# Transportation costs (per unit)
costs = {
    ('W1', 'A'): 2, ('W1', 'B'): 3, ('W1', 'C'): 2, ('W1', 'D'): 4,
    ('W2', 'A'): 4, ('W2', 'B'): 1, ('W2', 'C'): 5, ('W2', 'D'): 3,
    ('W3', 'A'): 5, ('W3', 'B'): 3, ('W3', 'C'): 4, ('W3', 'D'): 2
}

# Creating model
model = Model("Transportation")

# Decision variables
x = model.addVars(warehouses, stores, vtype=GRB.CONTINUOUS, name="x")

# Objective function: Minimize total shipping cost
model.setObjective(sum(costs[w, s] * x[w, s] for w in warehouses for s in stores), GRB.MINIMIZE)

# Supply constraints
for w in warehouses:
    model.addConstr(sum(x[w, s] for s in stores) <= supply[w], f"Supply_{w}")

# Demand constraints
for s in stores:
    model.addConstr(sum(x[w, s] for w in warehouses) == demand[s], f"Demand_{s}")

# Solving model
model.optimize()

# Printing results
if model.status == GRB.OPTIMAL:
    print("Optimal shipping plan:")
    for w in warehouses:
        for s in stores:
            if x[w, s].x > 0:
                print(f"Ship {x[w, s].x} units from {w} to {s}")
    print(f"Minimum total shipping cost: ${model.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xbe0b55d2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 7e+01]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+02   2.000000e+01   0.000000e+00      0s
       1    3.7000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.700000000e+02
Optimal shipping plan:
Ship 20.0 units from W1 to A
Ship 50.0 units from W1 to C
Ship 20.0 units from W2 to A
Ship 30.0 units from W2 to B
Ship 60.0 units from W3 to D

In [ ]:
# Question 6

# Defining the network nodes
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Defining the arcs and their capacities
edges = {
    ('A', 'B'): 10,
    ('B', 'C'): 5,
    ('B', 'D'): 8,
    ('C', 'D'): 2,
    ('C', 'E'): 2,
    ('D', 'E'): 5,
    ('D', 'F'): 2,
    ('E', 'G'): 4,
    ('E', 'F'): 6,
    ('F', 'G'): 6
}

# Creating a Gurobi model
model = Model("MaxFlow")

# Decision variables: Flow on each edge
flow = model.addVars(edges.keys(), lb=0, ub=edges, vtype=GRB.CONTINUOUS, name="flow")

# Objective: Maximize flow into destination node 'G'
model.setObjective(flow.sum('*', 'G'), GRB.MAXIMIZE)

# Flow conservation constraints (excluding source 'A' and sink 'G')
for node in nodes:
    if node not in ['A', 'G']:
        model.addConstr(flow.sum('*', node) == flow.sum(node, '*'), f"flow_conservation_{node}")

# Source constraint: Total outflow from 'A'
model.addConstr(flow.sum('A', '*') == flow.sum('*', 'G'), "source_sink_balance")

# Solving the model
model.optimize()

# Printing results
if model.status == GRB.OPTIMAL:
    print(f"Maximum number of cars that can drive from Santa Barbara to Disney Land: {model.objVal} (in thousands)")
    print("Optimal Transportation Plan:")
    for (i, j) in edges.keys():
        if flow[i, j].x > 0:
            print(f"Flow from {i} to {j}: {flow[i, j].x} (in thousands)")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 10 columns and 20 nonzeros
Model fingerprint: 0x9cdd7780
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.02s
Presolved: 4 rows, 7 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   3.000000e+00   0.000000e+00      0s
       2    9.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  9.000000000e+00
Maximum number of cars that can drive from Santa Barbara to Disney Land: 9.0 (in thousands)
Optimal Transportation Plan:
Flow from

In [ ]:
# Question 7

# Problem 1 data (warehouses 1,2,3; stores A,B,C,D)
supply = [20, 30, 25]       # Warehouse capacities
demand = [10, 15, 25, 25]  # Store demands
costs = [
    [8, 6, 10, 9],   # Warehouse 1 → Stores A,B,C,D
    [9, 12, 13, 7],  # Warehouse 2 → Stores A,B,C,D
    [14, 9, 16, 5]   # Warehouse 3 → Stores A,B,C,D
]

model = Model("DualTransportation")

# Dual variables: u_i (unrestricted), v_j (unrestricted)
u = model.addVars(3, vtype=GRB.CONTINUOUS, name="u")  # Removed lb=0
v = model.addVars(4, vtype=GRB.CONTINUOUS, name="v")   # Removed lb=0

# Objective: Maximize Σ(supply_i * u_i + demand_j * v_j)
model.setObjective(
    sum(u[i] * supply[i] for i in range(3)) + sum(v[j] * demand[j] for j in range(4)),
    GRB.MAXIMIZE
)

# Constraints: u_i + v_j ≤ c_ij for all routes
for i in range(3):
    for j in range(4):
        model.addConstr(u[i] + v[j] <= costs[i][j], name=f"dual_{i}_{j}")

model.optimize()

# Outputting results
if model.status == GRB.OPTIMAL:
    optimal_u = [round(u[i].x, 2) for i in range(3)]
    optimal_v = [round(v[j].x, 2) for j in range(4)]
    max_profit = round(model.objVal, 2)
    print(f"Optimal warehouse prices (u_i): {optimal_u}")
    print(f"Optimal store prices (v_j): {optimal_v}")
    print(f"Maximum profit: ${max_profit}")
else:
    print("No solution found. Check problem formulation.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 7 columns and 24 nonzeros
Model fingerprint: 0xa9494799
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 10 rows, 7 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.9527500e+02   1.700350e+01   0.000000e+00      0s
       7    6.1500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.150000000e+02
Optimal warehouse prices (u_i): [4.0, 7.0, 5.0]
Optimal store prices (v_j): [2.0, 2.0, 6.0, 0.0]
Maximum profit: $615.0


In [ ]:
# Questoion 8

# Defining nodes and edges with costs
nodes = ['SB', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'DL']
edges = {('SB', 'A'): 3, ('SB', 'B'): 2, ('A', 'C'): 5, ('B', 'C'): 2,
         ('B', 'D'): 3, ('C', 'E'): 4, ('D', 'F'): 6, ('E', 'G'): 3, ('F', 'G'): 2,
         ('G', 'DL'): 3}

# Creating Gurobi model
model = Model("Min-Cut")

# Decision variables
z = model.addVars(nodes, vtype=GRB.BINARY, name="z")  # Node side
y = model.addVars(edges.keys(), vtype=GRB.BINARY, name="y")  # Edge cut

# Source and sink constraints
model.addConstr(z['SB'] == 0)  # Santa Barbara side
model.addConstr(z['DL'] == 1)  # Disneyland side

# Cut constraints
for (i, j) in edges:
    model.addConstr(y[i, j] >= z[j] - z[i])

# Objective function: Minimize cut cost
model.setObjective(sum(edges[i, j] * y[i, j] for (i, j) in edges), GRB.MINIMIZE)

# Solving the model
model.optimize()

# Outputting results
if model.status == GRB.OPTIMAL:
    print(f"Minimum cut cost: {model.objVal}")
    print("Edges to cut:")
    for (i, j) in edges:
        if y[i, j].x > 0.5:
            print(f"Cut edge: {i}-{j} (cost: {edges[i, j]})")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 19 columns and 32 nonzeros
Model fingerprint: 0x6e31e3a5
Variable types: 0 continuous, 19 integer (19 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 12 rows and 19 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Minimum cut cost: 3.0
Edges to cut:
Cut edge: G-DL 